In [1]:
# Import the necessary libraries
import sys
import os
import pandas as pd
import warnings

# Add the src folder to the path
sys.path.append(os.path.abspath('..'))
warnings.filterwarnings("ignore")

# Import the classes
from src.data_tool.connect_wrds import get_wrds_data, get_security_daily
from src.data_tool.yahoo_finance import yahoo_finance
from src.data_tool.ploygon import polygon
from src.data_tool.financial_datasets import FinancialDatasets
from src.database_tool.connect_db import ConnectDB
from src.database_tool.create_table import TableCreator

# Initialize the classes
yf = yahoo_finance()
pl = polygon()
fd = FinancialDatasets()

db =ConnectDB()
engine = db.get_engine()


FINANCIAL_DATASETS_API_KEY is set
YAHOO_CONSUMER_KEY and YAHOO_CONSUMER_SECRET are set
YAHOO_APP_ID is set
POLYGON_API_KEY is set
DB_USER and DB_PASSWORD are set
DB_USER and DB_PASSWORD are set


In [2]:
def get_database_size(db_instance, database_name):
    """
    Retrieves the total size of the database.
    """
    sql_statement = f"""
    SELECT table_schema "database_name", 
           sum(data_length + index_length) / 1024 / 1024 "database_size_mb" 
    FROM information_schema.TABLES 
    WHERE table_schema = '{database_name}'
    GROUP BY table_schema;
    """
    result = db_instance.execute_sql(sql_statement)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    return df

database_name = 'Elite_Traders' # Replace with your database name if different
database_size_df = get_database_size(db, database_name)
print(database_size_df)

   database_name database_size_mb
0  Elite_Traders       0.89062500


In [3]:
def get_table_sizes(db_instance):
    """
    Retrieves the size of each table in the database.
    """
    sql_statement = "SHOW TABLE STATUS"
    result = db_instance.execute_sql(sql_statement)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    size_df = df[['Name', 'Data_length', 'Index_length', 'Data_free']]
    size_df['Total_size'] = size_df['Data_length'] + size_df['Index_length']
    size_df['Data_length_MB'] = size_df['Data_length'] / 1024 / 1024
    size_df['Index_length_MB'] = size_df['Index_length'] / 1024 / 1024
    size_df['Data_free_MB'] = size_df['Data_free'] / 1024 / 1024
    size_df['Total_size_MB'] = size_df['Total_size'] / 1024 / 1024
    return size_df

table_sizes_df = get_table_sizes(db)
print(table_sizes_df)

                         Name  Data_length  Index_length  Data_free  \
0      capital_structure_debt        16384             0          0   
1    capital_structure_equity        16384             0          0   
2   capital_structure_summary        16384             0          0   
3                company_news       114688             0          0   
4           financial_metrics        16384             0          0   
5            financial_ratios        16384             0          0   
6         fundamentals_annual        16384             0          0   
7      fundamentals_quarterly        49152             0          0   
8                 identifiers        16384             0          0   
9               insider_trade       163840             0          0   
10                      price        16384             0          0   
11             security_daily       475136             0          0   

    Total_size  Data_length_MB  Index_length_MB  Data_free_MB  Total_size_MB

In [4]:
db.show_tables()

,Tables
0,capital_structure_debt
1,capital_structure_equity
2,capital_structure_summary
3,company_news
4,financial_metrics
5,financial_ratios
6,fundamentals_annual
7,fundamentals_quarterly
8,identifiers
9,insider_trade


In [5]:
tc = TableCreator()
tc.create_all_tables()

DB_USER and DB_PASSWORD are set
Price table already exists
Financial Metrics table already exists
Insider Trade table already exists
Company News table already exists
All tables created successfully


In [6]:
db.show_tables()

,Tables
0,capital_structure_debt
1,capital_structure_equity
2,capital_structure_summary
3,company_news
4,financial_metrics
5,financial_ratios
6,fundamentals_annual
7,fundamentals_quarterly
8,identifiers
9,insider_trade


In [7]:
news = db.read_table('company_news')

In [8]:
news.tail()

,id,ticker,title,author,source,date,url,sentiment
276,277,NVDA,Quantum Leap: What a 40% CAGR Means for the Fu...,Prnewswire,Benzinga,2025-04-01T16:08:00Z,https://www.benzinga.com/pressreleases/25/04/n...,positive
277,278,NVDA,Where Will Super Micro Computer Stock Be in 1 ...,Leo Sun,The Motley Fool,2025-04-01T12:55:00Z,https://www.fool.com/investing/2025/04/01/wher...,positive
278,279,NVDA,If Nearly Half of S&P 500 Stocks Are Up in 202...,Daniel Foelber,The Motley Fool,2025-04-01T12:44:00Z,https://www.fool.com/investing/2025/04/01/stoc...,positive
279,280,NVDA,Nvidia and Broadcom Got Fantastic News From Op...,Jose Najarro,The Motley Fool,2025-04-01T12:30:00Z,https://www.fool.com/investing/2025/04/01/nvid...,neutral
280,281,NVDA,"Meet the Hard Asset That's Bigger Than Apple, ...",Bram Berkowitz,The Motley Fool,2025-04-01T12:05:00Z,https://www.fool.com/investing/2025/04/01/meet...,neutral


In [9]:
news['date'] = pd.to_datetime(news['date'])
news['date'] = news['date'].dt.strftime('%Y-%m-%d')
news['date'].unique()


array(['2025-03-31', '2025-03-30', '2025-03-29', '2025-03-27',
       '2025-03-26', '2025-03-25', '2025-03-24', '2025-03-23',
       '2025-03-22', '2025-03-21', '2025-03-20', '2025-03-19',
       '2025-03-18', '2025-03-17', '2025-03-16', '2025-03-15',
       '2025-03-14', '2025-03-13', '2025-03-12', '2025-03-11',
       '2025-03-10', '2025-03-09', '2025-03-08', '2025-03-07',
       '2025-03-06', '2025-03-05', '2025-03-04', '2025-03-03',
       '2025-03-02', '2025-03-01', '2025-04-02', '2025-04-01'],
      dtype=object)